<a href="https://colab.research.google.com/github/Modinat-A/Seq2Seq/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import json
import re
import py
from collections import Counter
import keras
from keras import layers, models
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import Input
from keras.layers import Embedding
from nltk.translate.bleu_score import sentence_bleu

In [85]:
# load training data and merge
file_name = open('train_spider.json')
v= pd.read_json(file_name)
train_data = v.head(1000)

# load testing data
file_name = open('dev.json')
c = pd.read_json(file_name)
test_data = c.head(1000)


In [86]:
train_data = train_data[['query','question']]

In [87]:
train_data.head()

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...


In [88]:
train_data['query']= train_data['query'].apply(lambda x: x.lower())
train_data['question'] = train_data['question'].apply(lambda x: x.lower())

In [89]:
train_data['question']=train_data['question'].apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
train_data['query'] =train_data['query'].apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [90]:
exclude = ['!','"','#','$','%','&',"'",'(',')', '*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','`','{','|','}','~']
train_data['question']=train_data['question'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
train_data['query']= train_data['query'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [91]:
train_data['query'] = train_data['query'].apply(lambda x : 'START_ '+ x + ' _END')

In [92]:
input_words=set()
target_words=set()
for i in train_data['question']:
    for word in i.split():
        if word not in input_words:
            input_words.add(word)
    
for i in train_data['query']:
    for word in i.split():
        if word not in target_words:
            target_words.add(word)

In [93]:
len(input_words), len(target_words)

(894, 798)

In [94]:
lenght_list=[]
for l in train_data['query']:
    lenght_list.append(len(l.split(' ')))
max_query = np.max(lenght_list)

In [95]:
lenght_list=[]
for l in train_data['question']:
    lenght_list.append(len(l.split(' ')))
max_question = np.max(lenght_list)

In [96]:
train_data['query']

0      START_ select count from head where age    56 ...
1      START_ select name  COMMA  born_state  COMMA  ...
2      START_ select creation  COMMA  name  COMMA  bu...
3      START_ select maxbudget_in_billions  COMMA  mi...
4      START_ select avgnum_employees from department...
                             ...                        
995    START_ select avgenrollment from university wh...
996    START_ select avgenrollment from university wh...
997    START_ select enrollment  COMMA  primary_confe...
998    START_ select enrollment  COMMA  primary_confe...
999    START_ select sumenrollment  COMMA  minenrollm...
Name: query, Length: 1000, dtype: object

In [97]:
train_data['question']

0      how many heads of the departments are older th...
1      list the name COMMA born state and age of the ...
2      list the creation year COMMA name and budget o...
3      what are the maximum and minimum budget of the...
4      what is the average number of employees of the...
                             ...                        
995    what are the average enrollment size of the un...
996    return the average enrollment of universities ...
997    show the enrollment and primary_conference of ...
998    what are the enrollment and primary conference...
999    what is the total and minimum enrollment of al...
Name: question, Length: 1000, dtype: object

In [123]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
# del all_eng_words, all_french_words

In [124]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [125]:
encoder_input_data = np.zeros((len(train_data['question']), max_question),dtype='float32')
decoder_input_data = np.zeros((len(train_data['query']), max_query),dtype='float32')
decoder_target_data = np.zeros((len(train_data['query']),max_query, num_decoder_tokens),dtype='float32')

In [126]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(1000, 26)
(1000, 71)
(1000, 71, 798)


In [127]:
for i, (input_text, target_text) in enumerate(zip(train_data['question'], train_data['query'])):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [137]:
embedding_size = 30

In [139]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(84, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(84, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [140]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.3)

Epoch 1/20
6/6 [==============================] - 7s 564ms/step - loss: 1.3472 - acc: 0.0213 - val_loss: 1.3004 - val_acc: 0.0279
Epoch 2/20
6/6 [==============================] - 2s 386ms/step - loss: 1.1874 - acc: 0.0277 - val_loss: 1.1676 - val_acc: 0.0324
Epoch 3/20
6/6 [==============================] - 2s 402ms/step - loss: 1.1134 - acc: 0.0318 - val_loss: 1.1430 - val_acc: 0.0316
Epoch 4/20
6/6 [==============================] - 2s 377ms/step - loss: 1.0940 - acc: 0.0318 - val_loss: 1.1325 - val_acc: 0.0320
Epoch 5/20
6/6 [==============================] - 2s 382ms/step - loss: 1.0837 - acc: 0.0321 - val_loss: 1.1268 - val_acc: 0.0390
Epoch 6/20
6/6 [==============================] - 2s 381ms/step - loss: 1.0765 - acc: 0.0360 - val_loss: 1.1208 - val_acc: 0.0390
Epoch 7/20
6/6 [==============================] - 2s 381ms/step - loss: 1.0707 - acc: 0.0380 - val_loss: 1.1167 - val_acc: 0.0393
Epoch 8/20
6/6 [==============================] - 2s 379ms/step - loss: 1.0669 - acc: 0.04

In [141]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_20 (Embedding)    (None, None, 30)          26820     
                                                                 
 lstm_20 (LSTM)              [(None, 84),              38640     
                              (None, 84),                        
                              (None, 84)]                        
                                                                 
Total params: 65,460
Trainable params: 65,460
Non-trainable params: 0
_________________________________________________________________


In [142]:
decoder_state_input_h = Input(shape=(84,))
decoder_state_input_c = Input(shape=(84,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [143]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > max_query):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [144]:
data = test_data['question'].apply(lambda x:x.lower())
for seq_index in [12,22,78,99]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    # print('-')
    print('Input sentence:', data[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)


Input sentence: 12    list all song names by singers above the avera...
Name: question, dtype: object
Decoded sentence:  select select from from from from from _END
Input sentence: 22    show the stadium name and the number of concer...
Name: question, dtype: object
Decoded sentence:  select from from from from _END
Input sentence: 78    what is the id of the pet owned by the student...
Name: question, dtype: object
Decoded sentence:  select from from from from _END
Input sentence: 99    find the name of the makers that produced some...
Name: question, dtype: object
Decoded sentence:  select select from from from from _END


Make Predictions

In [115]:
test_data['question']= test_data['question'].apply(lambda x:x.lower())
scores = []
for i,row in enumerate(data):
    input_seq = encoder_input_data[i: i + 1]
    decoded_sentence = decode_sequence(input_seq)
    score = sentence_bleu(decoded_sentence, row)
    scores.append(score)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Evaluate Performance

In [118]:
print('Bleu score',sum(scores)/len(scores))

Bleu score 0.62127492737195
